In [15]:
import os
import errno
import rasterio

# Lista de nombres de puertos
# nombres_de_puertos = ["TPCallao"]
nombres_de_puertos = ["TPPaita"]

# Directorio base de entrada para las imágenes TIFF
base_folder_tif = "/home/farix/python/dataset"

# Directorio base de salida para las imágenes PNG
base_folder_png = base_folder_tif  # Usamos el mismo directorio base

# Recorre los nombres de los puertos
for puerto in nombres_de_puertos:
    input_folder_tif = os.path.join(base_folder_tif, puerto, "BLOQUES512")
    output_base_folder_png = os.path.join(base_folder_png, puerto, "BLOQUES512PNG")

    # Verifica si el directorio de salida para PNG base existe, y si no, créalo
    if not os.path.exists(output_base_folder_png):
        os.makedirs(output_base_folder_png)

    # Recorre las carpetas en el directorio de entrada
    for carpeta in os.listdir(input_folder_tif):
        carpeta_tif = os.path.join(input_folder_tif, carpeta)
        
        

        # Verifica si la carpeta de imágenes PNG de salida existe, y si no, créala
        carpeta_png = os.path.join(output_base_folder_png, carpeta)
        if not os.path.exists(carpeta_png):
            os.makedirs(carpeta_png)
        else:
            # Si la carpeta ya existe, pasa al siguiente bucle
            continue
        # Recorre las imágenes TIFF en la carpeta de imágenes de entrada
        for imagen in os.listdir(carpeta_tif):
            imagen_tif_path = os.path.join(carpeta_tif, imagen)
            
            # Verifica si la extensión no es ".tif.aux.xml" antes de procesar
            if not imagen.endswith(".tif.aux.xml"):
                # Convierte la imagen TIFF a PNG
                
                # Abre la imagen TIFF con Rasterio
                imagen_tif = rasterio.open(imagen_tif_path)

                # Obtiene el valor mínimo y máximo de las bandas
                bandas = [imagen_tif.read(i) for i in range(1, imagen_tif.count + 1)]
                bandas_min = [banda.min() for banda in bandas]
                bandas_max = [banda.max() for banda in bandas]

                # Nombre del archivo de salida en formato PNG
                nombre_base = os.path.splitext(imagen)[0]
                imagen_png_path = os.path.join(carpeta_png, f"{nombre_base}.png")

                # Comando para convertir de TIFF a PNG utilizando gdal_translate
                comando_convertir = [
                    "gdal_translate",
                    "-ot", "Byte",
                    "-of", "PNG",
                    f"-scale_1", f"{bandas_min[0]}", f"{bandas_max[0]}",
                    f"-scale_2", f"{bandas_min[1]}", f"{bandas_max[1]}",
                    f"-scale_3", f"{bandas_min[2]}", f"{bandas_max[2]}",
                    "-a_nodata", "none",
                    imagen_tif_path, imagen_png_path
                ]

                # Ejecuta el comando para convertir a PNG
                os.system(" ".join(comando_convertir))

# Elimina los archivos XML generados durante la conversión
delete_xml = f"find {base_folder_png} -name '*.aux.xml' -exec rm {{}} +"
os.system(delete_xml)


0

In [16]:
import os

def sync_folders(png_folder, tif_folder):
    # Obtén la lista de carpetas en el directorio PNG
    png_subfolders = [f for f in os.listdir(png_folder) if os.path.isdir(os.path.join(png_folder, f))]

    for subfolder in png_subfolders:
        # Crea las rutas completas para las carpetas PNG y TIF correspondientes
        png_subfolder = os.path.join(png_folder, subfolder)
        tif_subfolder = os.path.join(tif_folder, subfolder)

        # Verifica si la carpeta TIF correspondiente existe
        if os.path.exists(tif_subfolder):
            # Obtén la lista de archivos PNG y TIF en las carpetas correspondientes
            png_files = [f for f in os.listdir(png_subfolder) if f.endswith(".png")]
            tif_files = [f for f in os.listdir(tif_subfolder) if f.endswith(".tif")]

            # Crea un conjunto de nombres base (sin extensiones) para los archivos PNG
            png_names = {os.path.splitext(file)[0] for file in png_files}

            # Crea un conjunto de nombres base (sin extensiones) para los archivos TIF
            tif_names = {os.path.splitext(file)[0] for file in tif_files}

            # Encuentra los nombres que existen en ambos conjuntos
            common_names = png_names.intersection(tif_names)

            # Elimina los archivos TIF que no tienen un equivalente en PNG
            for tif_file in tif_files:
                tif_name_base = os.path.splitext(tif_file)[0]
                if tif_name_base not in common_names:
                    tif_path = os.path.join(tif_subfolder, tif_file)
                    os.remove(tif_path)

            # Elimina los archivos PNG que no tienen un equivalente en TIF
            for png_file in png_files:
                png_name_base = os.path.splitext(png_file)[0]
                if png_name_base not in common_names:
                    png_path = os.path.join(png_subfolder, png_file)
                    os.remove(png_path)
        else:
            # Si la carpeta TIF correspondiente no existe, elimina la carpeta PNG
            os.rmdir(png_subfolder)

# Ejemplo de uso:
png_folder = "/home/farix/python/dataset/TPPaita/BLOQUES512PNG/"
tif_folder = "/home/farix/python/dataset/TPPaita/BLOQUES512/"
sync_folders(png_folder, tif_folder)


In [17]:
import os
# Sincronizar carpetas ya en formato recorrido
# Rutas de las carpetas
ruta_imagenes_tiff = '/home/farix/python/dataset/TPCallao/imagenesTIFF'
ruta_imagenes_png = '/home/farix/python/dataset/TPCallao/imagenespng'

# Obtener los nombres de los archivos en ambas carpetas
archivos_tiff = os.listdir(ruta_imagenes_tiff)
archivos_png = os.listdir(ruta_imagenes_png)

# Obtener los nombres base de los archivos sin la extensión
nombres_base_tiff = {os.path.splitext(archivo)[0] for archivo in archivos_tiff}
nombres_base_png = {os.path.splitext(archivo)[0] for archivo in archivos_png}

# Identificar archivos en imagenesTIFF que no tienen un equivalente en imagenespng
archivos_a_eliminar = [archivo for archivo in archivos_tiff if os.path.splitext(archivo)[0] not in nombres_base_png]

# Mostrar la cantidad de archivos a eliminar
print("Número de archivos a eliminar:", len(archivos_a_eliminar))
print( archivos_a_eliminar)

# Eliminar los archivos no coincidentes en imagenesTIFF
# for archivo in archivos_a_eliminar:
#     ruta_completa = os.path.join(ruta_imagenes_tiff, archivo)
#     os.remove(ruta_completa)

Número de archivos a eliminar: 0
[]


In [18]:
# Utilidad - Copiar todas las imagenes png a carpeta
import os
import shutil

# Ruta de origen
ruta_origen = '/home/farix/python/dataset/TPPaita/BLOQUES512PNG'

# Ruta de destino para los archivos PNG
ruta_destino_imagenes = '/home/farix/python/dataset/TPPaita/imagenespng'

# Crear la carpeta 'imagenespng' en caso de que no exista
if not os.path.exists(ruta_destino_imagenes):
    os.mkdir(ruta_destino_imagenes)

# Recorrer las carpetas en la ruta de origen
for dirpath, dirnames, filenames in os.walk(ruta_origen):
    for filename in filenames:
        if filename.lower().endswith('.png'):
            # Ruta completa del archivo PNG
            ruta_completa_imagen = os.path.join(dirpath, filename)
            
            # Copiar el archivo a la carpeta de destino
            shutil.copy(ruta_completa_imagen, ruta_destino_imagenes)


In [19]:
# Utilidad - Copiar todas las imagenes tiff a carpeta
import os
import shutil

# Ruta de origen
ruta_origen = '/home/farix/python/dataset/TPPaita/BLOQUES512'

# Ruta de destino para los archivos PNG
ruta_destino_imagenes = '/home/farix/python/dataset/TPPaita/imagenestiff'

# Crear la carpeta 'imagenespng' en caso de que no exista
if not os.path.exists(ruta_destino_imagenes):
    os.mkdir(ruta_destino_imagenes)

# Recorrer las carpetas en la ruta de origen
for dirpath, dirnames, filenames in os.walk(ruta_origen):
    for filename in filenames:
        if filename.lower().endswith('.tif'):
            # Ruta completa del archivo PNG
            ruta_completa_imagen = os.path.join(dirpath, filename)
            
            # Copiar el archivo a la carpeta de destino
            shutil.copy(ruta_completa_imagen, ruta_destino_imagenes)


In [22]:
imagen_tif_path ='/home/farix/python/dataset/TPPaita/BLOQUES/IMG_PER1_20200323155052_ORT_P_000054_pansharpened/IMG_PER1_20200323_block_2_0.tif'
imagen_tif = rasterio.open(imagen_tif_path)

# Obtiene el valor mínimo y máximo de las bandas
bandas = [imagen_tif.read(i) for i in range(1, imagen_tif.count + 1)]
bandas_min = [banda.min() for banda in bandas]
bandas_max = [banda.max() for banda in bandas]

# Nombre del archivo de salida en formato PNG
nombre_base = os.path.splitext(imagen)[0]
imagen_png_path = os.path.join(carpeta_png, f"{nombre_base}.png")

# Comando para convertir de TIFF a PNG utilizando gdal_translate
comando_convertir = [
"gdal_translate",
"-ot", "Byte",
"-of", "PNG",
f"-scale_1", f"{bandas_min[0]}", f"{bandas_max[0]}",
f"-scale_2", f"{bandas_min[1]}", f"{bandas_max[1]}",
f"-scale_3", f"{bandas_min[2]}", f"{bandas_max[2]}",
"-a_nodata", "none",
imagen_tif_path, imagen_png_path
]

# Ejecuta el comando para convertir a PNG
os.system(" ".join(comando_convertir))

RasterioIOError: /home/farix/python/dataset/TPPaita/BLOQUES/IMG_PER1_20200323155052_ORT_P_000054_pansharpened.tif: No such file or directory